## Reading Preprocessed data

In [18]:
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix,accuracy_score,roc_auc_score,roc_curve,auc,f1_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import GridSearchCV

In [19]:
train_df = pd.read_csv("/kaggle/input/yahoo-troll-question-detection/train_df.csv")
test_df = pd.read_csv("/kaggle/input/yahoo-troll-question-detection/test_df.csv")

In [20]:
test_df

,qid,question_text
0,a4f3da3a3df9dd881edd,My period is due on my wedding day. How can I ...
1,9914c62ed3f69684d549,How many numbers higher than a million can be ...
2,8138ae48649e37091a91,"How come I feel nothing for my family, but sti..."
3,981b4753d17ef14d09f7,"In case of collapse of the Democratic party, w..."
4,452e2c705276ba16b7b7,Who is Émile Naoumoff?
...,...,...
306117,a352dff4fcc2571815ce,Did anyone get an update on Maruti Suzuki All ...
306118,ad4a8498d97c536c67b9,What 5 people in history do you find the most ...
306119,19784a27b55d4b453fda,How can I remove the tan on my forehead?
306120,370191dba26465997879,"If you are a well known hacker, will you be mo..."


## Vectorization of the dataset

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack

# # from sklearn.feature_extraction.text import TfidfVectorizer

# # We can change the number of words counted together and see results on accuracy
# CountVectorizer(analyzer='word', ngram_range=(1, 3))

word_vectorizer = TfidfVectorizer(
    strip_accents = 'unicode',
    analyzer = 'word',
    ngram_range = (1, 3),
    max_df = 0.5,
    max_features = 150000,
)

char_vectorizer = TfidfVectorizer(
    strip_accents = 'unicode',
    analyzer = 'char',
    ngram_range = (1, 3),
    max_df = 0.5,
    max_features = 150000,
)

# # tfidf = TfidfVectorizer()

X_train1_wv = word_vectorizer.fit_transform(train_df.question_text.values.astype('U'))
# X_validate1_wv = word_vectorizer.transform(X_validate.values.astype('U'))

X_train1_cv = char_vectorizer.fit_transform(train_df.question_text.values.astype('U'))
# X_validate1_cv = char_vectorizer.transform(X_validate.values.astype('U'))
# # X_train = tfidf.fit(X_train)

train = hstack((X_train1_wv, X_train1_cv)).tocsr()
# X_validate1 = hstack((X_validate1_wv, X_validate1_cv)).tocsr()


X_test1_wv = word_vectorizer.transform(test_df.question_text)
X_test1_cv = char_vectorizer.transform(test_df.question_text)

test_df_matrix = hstack((X_test1_wv, X_test1_cv)).tocsr()


In [22]:
# tfidf = TfidfVectorizer(ngram_range=(1,3))
# # Numericalize the train dataset
# train = tfidf.fit_transform(train_df.question_text.values.astype('U'))

# # for test_df
# test_df_matrix = tfidf.transform(test_df.question_text.values.astype('U'))
# # print(test_df_matrix.shape)
# test_df_matrix

In [23]:
train

<1000000x232101 sparse matrix of type '<class 'numpy.float64'>'
	with 127658239 stored elements in Compressed Sparse Row format>

## Doing train test split of the train_df.csv

In [24]:

X_train, X_test, y_train, y_test = train_test_split(
    train, train_df.target.values, test_size=0.20)

X_train.shape
train = X_train
test = X_test
y_test
# train=train
# y_train=train_df.target

array([0, 0, 0, ..., 0, 0, 0])

In [25]:
train.shape

(800000, 232101)

In [26]:
# from sklearn.linear_model import LogisticRegression 
# model = LogisticRegression(dual = False,
#     class_weight = {0: 0.9, 1: 2},max_iter=10000)
# # parameters = {'class_weight':[{0:0.2, 1:0.8},{0:0.25,1:0.75},{0:0.3, 1:0.7}]}
# # model=GridSearchCV(model1,parameters,verbose=4,cv=2,scoring='f1')
# model.fit(train,y_train)
# y_pred = model.predict(train)
# print("train f1 score: ",f1_score(y_train,y_pred))
# print("test f1 score: ",f1_score(y_test,model.predict(test)))
# test_y_pred = model.predict(test_df_matrix)
# print(metrics.classification_report(y_train,y_pred) )
# # print(model.predict_proba(train))

In [33]:
model1 = LogisticRegression()
parameters = {'class_weight':[{0:0.1, 1:1},{0:0.1,1:2},{0:0.1, 1:3},{0:0.1, 1:4}]}
model=GridSearchCV(model1,parameters,verbose=10,cv=2,scoring='f1')
model.fit(train,y_train)

Fitting 2 folds for each of 4 candidates, totalling 8 fits
[CV 1/2; 1/4] START class_weight={0: 0.1, 1: 1}.................................
[CV 1/2; 1/4] END ..class_weight={0: 0.1, 1: 1};, score=0.571 total time=  52.3s
[CV 2/2; 1/4] START class_weight={0: 0.1, 1: 1}.................................
[CV 2/2; 1/4] END ..class_weight={0: 0.1, 1: 1};, score=0.570 total time=  46.8s
[CV 1/2; 2/4] START class_weight={0: 0.1, 1: 2}.................................


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


[CV 1/2; 2/4] END ..class_weight={0: 0.1, 1: 2};, score=0.505 total time= 1.1min
[CV 2/2; 2/4] START class_weight={0: 0.1, 1: 2}.................................
[CV 2/2; 2/4] END ..class_weight={0: 0.1, 1: 2};, score=0.505 total time=  55.6s
[CV 1/2; 3/4] START class_weight={0: 0.1, 1: 3}.................................


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


[CV 1/2; 3/4] END ..class_weight={0: 0.1, 1: 3};, score=0.467 total time= 1.0min
[CV 2/2; 3/4] START class_weight={0: 0.1, 1: 3}.................................


...

[CV 1/2; 4/4] END ..class_weight={0: 0.1, 1: 4};, score=0.441 total time= 1.1min
[CV 2/2; 4/4] START class_weight={0: 0.1, 1: 4}.................................


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


[CV 2/2; 4/4] END ..class_weight={0: 0.1, 1: 4};, score=0.443 total time= 1.1min


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


GridSearchCV(cv=2, estimator=LogisticRegression(),
             param_grid={'class_weight': [{0: 0.1, 1: 1}, {0: 0.1, 1: 2},
                                          {0: 0.1, 1: 3}, {0: 0.1, 1: 4}]},
             scoring='f1', verbose=10)

In [41]:
model.best_estimator_

LogisticRegression(class_weight={0: 0.1, 1: 1})

In [34]:
# def custom_predict(X, threshold):
#     probs = model.predict_proba(X) 
#     return (probs[:, 1] > threshold).astype(int)
    
# Y_pred = custom_predict(train,0.4)
# print("train f1 score: ",f1_score(y_train,Y_pred))

In [35]:
# test_y_pred = custom_predict(X=test_df_matrix,threshold=0.4)

In [36]:
pred_proba_df = pd.DataFrame(model.predict_proba(train))
threshold_list = [0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,.7,.75,.8,.85,.9,.95,.99]
for i in threshold_list:
    print ('\n******** For i = {} ******'.format(i))
    Y_test_pred = [1 if j > i else 0 for j in pred_proba_df[1]]
#     print((Y_test_pred))
#     print(pred_proba_df[1])
#     print((Y_test_pred))
#     print(y_test.shape)
#     print(np.array(Y_test_pred).shape)
    
    test_accuracy = metrics.f1_score(y_train,Y_test_pred)
    print(test_accuracy)
#                                          y_train_pred.iloc[:,1].as_matrix().reshape(y_train_pred.iloc[:,1].size,1))
#     print('Our testing accuracy is {}'.format(test_accuracy))

#     print(confusion_matrix(y_train.reshape(y_train.as_matrix().size,1),
#                            y_test_pred.iloc[:,1].as_matrix().reshape(y_test_pred.iloc[:,1].as_matrix().size,1)))


******** For i = 0.05 ******
0.2037405606107071

******** For i = 0.1 ******
0.2823060832688616

******** For i = 0.15 ******
0.34585317733911775

******** For i = 0.2 ******
0.39940062325301845

******** For i = 0.25 ******
0.44625866758254507

******** For i = 0.3 ******
0.4877713347359778

******** For i = 0.35 ******
0.5253391714306511

******** For i = 0.4 ******
0.557880038927205

******** For i = 0.45 ******
0.5860982943646638

******** For i = 0.5 ******
0.6103475202299756

******** For i = 0.55 ******
0.6318196859947789

******** For i = 0.6 ******
0.6488661130684081

******** For i = 0.65 ******
0.6597354282060437

******** For i = 0.7 ******
0.6644960938536809

******** For i = 0.75 ******
0.6617711073679429

******** For i = 0.8 ******
0.6489748407123019

******** For i = 0.85 ******
0.618662255208799

******** For i = 0.9 ******
0.5583950740889068

******** For i = 0.95 ******
0.425197545578952

******** For i = 0.99 ******
0.13631048499359652


In [37]:
# y_test_pred = [1 if j > 0.05 else 0 for j in pred_proba_df[1]]
y_test_pred=[]
pred_proba_df = pd.DataFrame(model.predict_proba(test))
# for i in range(len(pred_proba_df)):
#     if(pred_proba_df[1][i] > 0.45):
#         y_test_pred.append(1)
#     else:
#         y_test_pred.append(0)

# test_accuracy = metrics.f1_score(y_test,y_test_pred)
# print(test_accuracy)
threshold_list = [0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,.7,.75,.8,.85,.9,.95,.99]
for i in threshold_list:
    print ('\n******** For i = {} ******'.format(i))
    Y_test_pred = [1 if j > i else 0 for j in pred_proba_df[1]]
#     print((Y_test_pred))
#     print(pred_proba_df[1])
#     print((Y_test_pred))
#     print(y_test.shape)
#     print(np.array(Y_test_pred).shape)
    
    test_accuracy = metrics.f1_score(y_test,Y_test_pred)
    print(test_accuracy)
#                                  



# print(y_test_pred)


******** For i = 0.05 ******
0.2005307488173532

******** For i = 0.1 ******
0.275413211323992

******** For i = 0.15 ******
0.3355412753336629

******** For i = 0.2 ******
0.38653494087183127

******** For i = 0.25 ******
0.4301528140141633

******** For i = 0.3 ******
0.46788688267787937

******** For i = 0.35 ******
0.5028066457765054

******** For i = 0.4 ******
0.5328486740991926

******** For i = 0.45 ******
0.5604787904241916

******** For i = 0.5 ******
0.581929469901168

******** For i = 0.55 ******
0.601010554010823

******** For i = 0.6 ******
0.6141792470858016

******** For i = 0.65 ******
0.6242387332521316

******** For i = 0.7 ******
0.6291039115830533

******** For i = 0.75 ******
0.6287720923053175

******** For i = 0.8 ******
0.6207068399850493

******** For i = 0.85 ******
0.5947685751847318

******** For i = 0.9 ******
0.541683569979716

******** For i = 0.95 ******
0.41585336824565583

******** For i = 0.99 ******
0.13948273284496934


In [38]:
# # y_test_pred = [1 if j > 0.05 else 0 for j in pred_proba_df[1]]
# y_test_pred=[]
# pred_proba_df = pd.DataFrame(model.predict_proba(test_df_matrix))
# # lund = pred_proba_df.to_numpy().reshape(2,500000)
# # print(lund.shape)
# # print(pred_proba_df)
# # print(pred_proba_df[0][3])
# for i in range(len(pred_proba_df)):
#     if(pred_proba_df[1][i] > 0.45):
#         y_test_pred.append(1)
#     else:
#         y_test_pred.append(0)

# # test_accuracy = metrics.f1_score(y_test,y_test_pred)
# # print(test_accuracy)
# # print(y_test_pred)
# # y_test_pred

In [39]:
# print(metrics.classification_report(y_train,y_pred) )

In [40]:
# Test_DF_TARGET = pd.DataFrame(test_y_pred,columns=['target'])
# TEST_DF_QID = pd.DataFrame(test_df ,columns=['qid'])
# TEST_DF = pd.concat([TEST_DF_QID, Test_DF_TARGET], axis=1, join='inner')
# TEST_DF.to_csv("sample_submission69.csv",index=False)